In [15]:
import sys,os
os.environ["JAX_PLATFORMS"] = "cpu"
from LLM.utils import *
from LLM.LLM_paths import *
from dadapy.hamming import Hamming
import torch
from time import time
torch.set_printoptions(precision=32,sci_mode=False)
sys.path.append('../../')
from paths import *

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f'{device=}')

layer_id = 0
sub_length = 120

Ntokens = sub_length
N_batches = 1 #50
max_length = 400
corpus = 'Wikitext'
LLM = 'Pythia'
randomize = 0
Lconcat = 150
batch_randomize = 1
sublength_cutoff = 300
Nbits = 1
batch_size = 100

act_outputfolder0 = get_act_outputfolder0(max_length,corpus,LLM,randomize,Lconcat,batch_randomize)
act_outputfolder = act_outputfolder0 + f'sub_length{sublength_cutoff:d}/'
sigmas = load_activations(N_batches,
                          act_outputfolder,
                          layer_id,
                          sub_length=sub_length, # 0 means every token
                          ).numpy()
Ns0 = sigmas.shape[0]
sigmas = formatting_activations(sigmas,sub_length,Ns0,layer_normalize=0)
sigmas = np.round(np.sign(sigmas),1)
sigmas[np.where(np.isclose(sigmas,0.))] = -1 # just in case


importing took 0.0 m
a.shape=torch.Size([100, 120, 1024])
a.shape=(100, 120, 1024)


In [16]:
histfolder = makefolder(base='results/BID/dbg/',
                        create_folder=True,
                        LLM=LLM,
                        corpus=corpus,
                        layer_id=layer_id,
                        sub_length=sub_length,
                        Nbits=Nbits,
                        batch_randomize=batch_randomize,
                        Ns=N_batches*batch_size,
                        )
H = Hamming(coordinates=sigmas)
start = time()
H.compute_distances()
H.D_histogram(compute_flag=1,
              save=True,
              resultsfolder=histfolder,
              )
print(f'{layer_id=:d} took {(time()-start)/60:.1f} min')

# if remove_spins:
#   for t in range(sublength_cutoff):
#     sigmas_filename = get_sigmas_filename(sigmasfolder0,sublength_cutoff,layer_id,t)
#     os.system(f'rm -f {sigmas_filename}')


layer_id=0 took 0.0 min
